In [2]:
!pip install num2words

     -------------------------------------- 143.3/143.3 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=b0012e46a96c53e76c8483862eec25398fb3026a96ef43cd8a3002fe4418837c
  Stored in directory: c:\users\kaout\appdata\local\pip\cache\wheels\70\4a\46\1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built docopt


In [4]:
!pip install tiktoken

     -------------------------------------- 786.4/786.4 kB 2.8 MB/s eta 0:00:00


In [6]:
!pip install openai

     -------------------------------------- 225.4/225.4 kB 2.3 MB/s eta 0:00:00
     -------------------------------------- 381.9/381.9 kB 7.9 MB/s eta 0:00:00
     ---------------------------------------- 75.9/75.9 kB 4.4 MB/s eta 0:00:00
     ---------------------------------------- 76.9/76.9 kB 4.4 MB/s eta 0:00:00
     ---------------------------------------- 58.3/58.3 kB ? eta 0:00:00
     ---------------------------------------- 1.9/1.9 MB 12.0 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.3.0
    Uninstalling typing_extensions-4.3.0:
      Successfully uninstalled typing_extensions-4.3.0


In [7]:
import os
import re
import requests
import sys
from num2words import num2words
import os
import pandas as pd
import numpy as np
import tiktoken
from openai import AzureOpenAI

In [16]:
df = pd.read_excel('DataHackathon.xlsx')
df

,Unnamed: 0,Y,Domain,area,keywords,Abstract,Cleaned_keywords,Cleaned_abstract,abstract
0,0,12,CS,Symbolic computation,(2+1)-dimensional non-linear optical waves; e...,(2 + 1)-dimensional non-linear optical waves t...,"['dimensional', 'non', 'linear', 'optical', 'w...","['dimensional', 'non', 'linear', 'optical', 'w...",(2 + 1)-dimensional non-linear optical waves t...
1,1,0,CS,Computer vision,tea category identification; computer vision;...,(Objective) In order to increase classificatio...,"['tea', 'category', 'identification', 'compute...","['objective', 'order', 'increase', 'classifica...",(objective) in order to increase classificatio...
2,2,5,CS,Computer graphics,3d images; virtual reality; smoothness,[Purpose] This study verified that the smoothn...,"['image', 'virtual', 'reality', 'smoothness']","['purpose', 'study', 'verify', 'smoothness', '...",[purpose] this study verified that the smoothn...
3,3,4,CS,Operating systems,biodiversity surveys; camera trapping; data m...,1. Camera trapping is a widely applied method ...,"['biodiversity', 'survey', 'camera', 'trap', '...","['camera', 'trap', 'widely', 'apply', 'method'...",1. camera trapping is a widely applied method ...
4,4,1,CS,Machine learning,distribution; endangered species; groundwater...,1. Species distribution modelling can be usefu...,"['distribution', 'endanger', 'species', 'groun...","['species', 'distribution', 'model', 'useful',...",1. species distribution modelling can be usefu...
...,...,...,...,...,...,...,...,...,...
6509,6509,3,CS,Cryptography,zero-difference balanced (zdb) function; gene...,Zero-difference balanced (ZDB) functions have ...,"['zero', 'difference', 'balance', 'zdb', 'func...","['zero', 'difference', 'balance', 'zdb', 'func...",zero-difference balanced (zdb) functions have ...
6510,6510,15,CS,Data structures,data structures for logic synthesis; error re...,Zero-Suppressed Binary Decision Diagrams (ZDDs...,"['data', 'structure', 'logic', 'synthesis', 'e...","['zero', 'suppress', 'binary', 'decision', 'di...",zero-suppressed binary decision diagrams (zdds...
6511,6511,15,CS,Data structures,linear programming; zinc; solver independence...,Zinc is a modeling language in which a concept...,"['linear', 'program', 'zinc', 'solver', 'indep...","['zinc', 'model', 'language', 'conceptual', 'm...",zinc is a modeling language in which a concept...
6512,6512,15,CS,Data structures,z-wave; wireless; embedded systems; internet ...,Z-Wave is an implementation of home automation...,"['z', 'wave', 'wireless', 'embed', 'systems', ...","['z', 'wave', 'implementation', 'home', 'autom...",z-wave is an implementation of home automation...


In [18]:
print(df.columns)


Index(['Unnamed: 0', 'Y', 'Domain', 'area', 'keywords', 'Abstract',
       'Cleaned_keywords', 'Cleaned_abstract', 'abstract'],
      dtype='object')


In [20]:
df_cleaned = df[['Cleaned_keywords', 'Cleaned_abstract']]

In [21]:
df_cleaned

,Cleaned_keywords,Cleaned_abstract
0,"['dimensional', 'non', 'linear', 'optical', 'w...","['dimensional', 'non', 'linear', 'optical', 'w..."
1,"['tea', 'category', 'identification', 'compute...","['objective', 'order', 'increase', 'classifica..."
2,"['image', 'virtual', 'reality', 'smoothness']","['purpose', 'study', 'verify', 'smoothness', '..."
3,"['biodiversity', 'survey', 'camera', 'trap', '...","['camera', 'trap', 'widely', 'apply', 'method'..."
4,"['distribution', 'endanger', 'species', 'groun...","['species', 'distribution', 'model', 'useful',..."
...,...,...
6509,"['zero', 'difference', 'balance', 'zdb', 'func...","['zero', 'difference', 'balance', 'zdb', 'func..."
6510,"['data', 'structure', 'logic', 'synthesis', 'e...","['zero', 'suppress', 'binary', 'decision', 'di..."
6511,"['linear', 'program', 'zinc', 'solver', 'indep...","['zinc', 'model', 'language', 'conceptual', 'm..."
6512,"['z', 'wave', 'wireless', 'embed', 'systems', ...","['z', 'wave', 'implementation', 'home', 'autom..."


In [23]:
!pip install sentence-transformers pandas openpyxl

     ---------------------------------------- 86.0/86.0 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 8.2/8.2 MB 12.2 MB/s eta 0:00:00
     -------------------------------------- 192.2/192.2 MB 1.8 MB/s eta 0:00:00
     ---------------------------------------- 1.1/1.1 MB 2.5 MB/s eta 0:00:00
     -------------------------------------- 977.6/977.6 kB 2.2 MB/s eta 0:00:00
     -------------------------------------- 330.1/330.1 kB 2.3 MB/s eta 0:00:00
     -------------------------------------- 169.0/169.0 kB 3.4 MB/s eta 0:00:00
     -------------------------------------- 277.8/277.8 kB 3.4 MB/s eta 0:00:00
     ---------------------------------------- 2.2/2.2 MB 3.2 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125925 sha256=f37892d4a6ede285e9883934395ebddaabec51e5de01ed5599b649ab20d64cb4
  

In [24]:
from sentence_transformers import SentenceTransformer
import pandas as pd


In [26]:
#Téléchargement du modèle
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')


.gitattributes:   0%|          | 0.00/690 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.69k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [27]:
df

,Unnamed: 0,Y,Domain,area,keywords,Abstract,Cleaned_keywords,Cleaned_abstract,abstract
0,0,12,CS,Symbolic computation,(2+1)-dimensional non-linear optical waves; e...,(2 + 1)-dimensional non-linear optical waves t...,"['dimensional', 'non', 'linear', 'optical', 'w...","['dimensional', 'non', 'linear', 'optical', 'w...",(2 + 1)-dimensional non-linear optical waves t...
1,1,0,CS,Computer vision,tea category identification; computer vision;...,(Objective) In order to increase classificatio...,"['tea', 'category', 'identification', 'compute...","['objective', 'order', 'increase', 'classifica...",(objective) in order to increase classificatio...
2,2,5,CS,Computer graphics,3d images; virtual reality; smoothness,[Purpose] This study verified that the smoothn...,"['image', 'virtual', 'reality', 'smoothness']","['purpose', 'study', 'verify', 'smoothness', '...",[purpose] this study verified that the smoothn...
3,3,4,CS,Operating systems,biodiversity surveys; camera trapping; data m...,1. Camera trapping is a widely applied method ...,"['biodiversity', 'survey', 'camera', 'trap', '...","['camera', 'trap', 'widely', 'apply', 'method'...",1. camera trapping is a widely applied method ...
4,4,1,CS,Machine learning,distribution; endangered species; groundwater...,1. Species distribution modelling can be usefu...,"['distribution', 'endanger', 'species', 'groun...","['species', 'distribution', 'model', 'useful',...",1. species distribution modelling can be usefu...
...,...,...,...,...,...,...,...,...,...
6509,6509,3,CS,Cryptography,zero-difference balanced (zdb) function; gene...,Zero-difference balanced (ZDB) functions have ...,"['zero', 'difference', 'balance', 'zdb', 'func...","['zero', 'difference', 'balance', 'zdb', 'func...",zero-difference balanced (zdb) functions have ...
6510,6510,15,CS,Data structures,data structures for logic synthesis; error re...,Zero-Suppressed Binary Decision Diagrams (ZDDs...,"['data', 'structure', 'logic', 'synthesis', 'e...","['zero', 'suppress', 'binary', 'decision', 'di...",zero-suppressed binary decision diagrams (zdds...
6511,6511,15,CS,Data structures,linear programming; zinc; solver independence...,Zinc is a modeling language in which a concept...,"['linear', 'program', 'zinc', 'solver', 'indep...","['zinc', 'model', 'language', 'conceptual', 'm...",zinc is a modeling language in which a concept...
6512,6512,15,CS,Data structures,z-wave; wireless; embedded systems; internet ...,Z-Wave is an implementation of home automation...,"['z', 'wave', 'wireless', 'embed', 'systems', ...","['z', 'wave', 'implementation', 'home', 'autom...",z-wave is an implementation of home automation...


In [28]:
# Generate text embeddings for the entire DataFrame
df['keywords_embeddings'] = df['Cleaned_keywords'].apply(lambda x: model.encode([x])[0])


In [29]:
df

,Unnamed: 0,Y,Domain,area,keywords,Abstract,Cleaned_keywords,Cleaned_abstract,abstract,keywords_embeddings
0,0,12,CS,Symbolic computation,(2+1)-dimensional non-linear optical waves; e...,(2 + 1)-dimensional non-linear optical waves t...,"['dimensional', 'non', 'linear', 'optical', 'w...","['dimensional', 'non', 'linear', 'optical', 'w...",(2 + 1)-dimensional non-linear optical waves t...,"[0.16464579, 0.0043636593, -0.16133256, 0.1538..."
1,1,0,CS,Computer vision,tea category identification; computer vision;...,(Objective) In order to increase classificatio...,"['tea', 'category', 'identification', 'compute...","['objective', 'order', 'increase', 'classifica...",(objective) in order to increase classificatio...,"[0.066833876, -0.64726573, -0.23238525, -0.375..."
2,2,5,CS,Computer graphics,3d images; virtual reality; smoothness,[Purpose] This study verified that the smoothn...,"['image', 'virtual', 'reality', 'smoothness']","['purpose', 'study', 'verify', 'smoothness', '...",[purpose] this study verified that the smoothn...,"[0.3983092, -0.033099394, 0.20134337, -0.26734..."
3,3,4,CS,Operating systems,biodiversity surveys; camera trapping; data m...,1. Camera trapping is a widely applied method ...,"['biodiversity', 'survey', 'camera', 'trap', '...","['camera', 'trap', 'widely', 'apply', 'method'...",1. camera trapping is a widely applied method ...,"[0.6441455, -0.04289675, 0.003967761, -0.03346..."
4,4,1,CS,Machine learning,distribution; endangered species; groundwater...,1. Species distribution modelling can be usefu...,"['distribution', 'endanger', 'species', 'groun...","['species', 'distribution', 'model', 'useful',...",1. species distribution modelling can be usefu...,"[0.58606786, -0.3721443, 0.48551887, 0.0618867..."
...,...,...,...,...,...,...,...,...,...,...
6509,6509,3,CS,Cryptography,zero-difference balanced (zdb) function; gene...,Zero-difference balanced (ZDB) functions have ...,"['zero', 'difference', 'balance', 'zdb', 'func...","['zero', 'difference', 'balance', 'zdb', 'func...",zero-difference balanced (zdb) functions have ...,"[0.14892437, -0.104249, -0.47876185, -0.168159..."
6510,6510,15,CS,Data structures,data structures for logic synthesis; error re...,Zero-Suppressed Binary Decision Diagrams (ZDDs...,"['data', 'structure', 'logic', 'synthesis', 'e...","['zero', 'suppress', 'binary', 'decision', 'di...",zero-suppressed binary decision diagrams (zdds...,"[-0.06576707, -0.14895836, -0.28880042, -0.114..."
6511,6511,15,CS,Data structures,linear programming; zinc; solver independence...,Zinc is a modeling language in which a concept...,"['linear', 'program', 'zinc', 'solver', 'indep...","['zinc', 'model', 'language', 'conceptual', 'm...",zinc is a modeling language in which a concept...,"[-0.09933168, -0.1892926, -0.28262392, -0.0863..."
6512,6512,15,CS,Data structures,z-wave; wireless; embedded systems; internet ...,Z-Wave is an implementation of home automation...,"['z', 'wave', 'wireless', 'embed', 'systems', ...","['z', 'wave', 'implementation', 'home', 'autom...",z-wave is an implementation of home automation...,"[-0.07542978, -0.36848098, -0.060247846, -0.51..."


In [30]:
# Generate text embeddings for the entire DataFrame
df['abstract_embeddings'] = df['Cleaned_abstract'].apply(lambda x: model.encode([x])[0])


In [31]:
df

,Unnamed: 0,Y,Domain,area,keywords,Abstract,Cleaned_keywords,Cleaned_abstract,abstract,keywords_embeddings,abstract_embeddings
0,0,12,CS,Symbolic computation,(2+1)-dimensional non-linear optical waves; e...,(2 + 1)-dimensional non-linear optical waves t...,"['dimensional', 'non', 'linear', 'optical', 'w...","['dimensional', 'non', 'linear', 'optical', 'w...",(2 + 1)-dimensional non-linear optical waves t...,"[0.16464579, 0.0043636593, -0.16133256, 0.1538...","[0.25874, -0.0706107, -0.39470172, 0.13750248,..."
1,1,0,CS,Computer vision,tea category identification; computer vision;...,(Objective) In order to increase classificatio...,"['tea', 'category', 'identification', 'compute...","['objective', 'order', 'increase', 'classifica...",(objective) in order to increase classificatio...,"[0.066833876, -0.64726573, -0.23238525, -0.375...","[0.25143936, -0.13442579, -0.30107194, -0.0610..."
2,2,5,CS,Computer graphics,3d images; virtual reality; smoothness,[Purpose] This study verified that the smoothn...,"['image', 'virtual', 'reality', 'smoothness']","['purpose', 'study', 'verify', 'smoothness', '...",[purpose] this study verified that the smoothn...,"[0.3983092, -0.033099394, 0.20134337, -0.26734...","[0.6832971, -0.19487655, 0.014208764, -0.07909..."
3,3,4,CS,Operating systems,biodiversity surveys; camera trapping; data m...,1. Camera trapping is a widely applied method ...,"['biodiversity', 'survey', 'camera', 'trap', '...","['camera', 'trap', 'widely', 'apply', 'method'...",1. camera trapping is a widely applied method ...,"[0.6441455, -0.04289675, 0.003967761, -0.03346...","[0.6579913, -0.16105512, -0.41694072, -0.16902..."
4,4,1,CS,Machine learning,distribution; endangered species; groundwater...,1. Species distribution modelling can be usefu...,"['distribution', 'endanger', 'species', 'groun...","['species', 'distribution', 'model', 'useful',...",1. species distribution modelling can be usefu...,"[0.58606786, -0.3721443, 0.48551887, 0.0618867...","[0.4396127, -0.11361948, 0.4218354, 0.12182456..."
...,...,...,...,...,...,...,...,...,...,...,...
6509,6509,3,CS,Cryptography,zero-difference balanced (zdb) function; gene...,Zero-difference balanced (ZDB) functions have ...,"['zero', 'difference', 'balance', 'zdb', 'func...","['zero', 'difference', 'balance', 'zdb', 'func...",zero-difference balanced (zdb) functions have ...,"[0.14892437, -0.104249, -0.47876185, -0.168159...","[0.117755905, -0.06222526, -0.52387154, -0.280..."
6510,6510,15,CS,Data structures,data structures for logic synthesis; error re...,Zero-Suppressed Binary Decision Diagrams (ZDDs...,"['data', 'structure', 'logic', 'synthesis', 'e...","['zero', 'suppress', 'binary', 'decision', 'di...",zero-suppressed binary decision diagrams (zdds...,"[-0.06576707, -0.14895836, -0.28880042, -0.114...","[0.07699966, -0.043052435, -0.28525674, -0.355..."
6511,6511,15,CS,Data structures,linear programming; zinc; solver independence...,Zinc is a modeling language in which a concept...,"['linear', 'program', 'zinc', 'solver', 'indep...","['zinc', 'model', 'language', 'conceptual', 'm...",zinc is a modeling language in which a concept...,"[-0.09933168, -0.1892926, -0.28262392, -0.0863...","[0.2566074, -0.21642117, -0.0021322202, -0.071..."
6512,6512,15,CS,Data structures,z-wave; wireless; embedded systems; internet ...,Z-Wave is an implementation of home automation...,"['z', 'wave', 'wireless', 'embed', 'systems', ...","['z', 'wave', 'implementation', 'home', 'autom...",z-wave is an implementation of home automation...,"[-0.07542978, -0.36848098, -0.060247846, -0.51...","[0.47278863, -0.19638063, -0.053334765, -0.467..."


In [32]:
from ast import literal_eval

In [33]:
from sklearn.manifold import TSNE

ValueError: malformed node or string: array([ 0.16464579,  0.00436366, -0.16133256,  0.15384264, -0.10033663,
       -0.31217948,  0.3565152 , -0.07447708,  0.12398057, -0.06557291,
        0.20884524, -0.16658863,  0.28581402,  0.1757283 ,  0.24025829,
        0.16483258, -0.14862953,  0.17256267, -0.6535579 ,  0.112871  ,
        0.06556047, -0.33126575, -0.00465768, -0.21817946, -0.14301787,
        0.41433617,  0.4407146 , -0.0715921 ,  0.13782525,  0.42684197,
        0.19954339,  0.63384277, -0.02579894,  0.15159039, -0.3888747 ,
       -0.1807324 , -0.200701  ,  0.13384089,  0.02597547, -0.4417947 ,
       -0.09870802,  0.33924255,  0.34217632,  0.00905481,  0.12653615,
       -0.23981881, -0.18532026,  0.03883857, -0.12396704, -0.53188026,
       -0.47982934, -0.13863492, -0.47569793,  0.16903698,  0.00605334,
        0.03900977, -0.04978579, -0.40854394,  0.0224627 , -0.62997687,
        0.30975387, -0.06537962, -0.15584154,  0.37574843,  0.32688975,
        0.13284191, -0.284

In [ ]:
output_xlsx_path = 'Hackathon2023_DataEnbedding'
df.to_excel(output_xlsx_path, index=False, engine='openpyxl')